## Objective

To match item names with the item descriptions in MMIC data using fuzzy logic

In [67]:
import pandas as pd
import numpy as np
from pyreadr import read_r
import sys
sys.path.append("/home/jupyter/H2O_GLM_IMPLEMENTATION/")
sys.path.append("/home/jupyter/H2O_GLM_IMPLEMENTATION/fns/")
sys.path.append("/home/jupyter/DOTCOM_INTEGRATION/fns/")
sys.path.append("/home/jupyter/Admin_Items/")
import config
from predict_fns import *
from text_cleaning_fns import *
from match_fns import *
import pickle
from tqdm.notebook import tqdm
tqdm.pandas()
from fuzzywuzzy import fuzz, process
import time
from pandarallel import pandarallel
pandarallel.initialize(nb_workers = 11, progress_bar = True, use_memory_fs = False)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

INFO: Pandarallel will run on 11 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [68]:
from fuzzy_fns import *

In [69]:
partial = config.partial
ratio_type = config.ratio_type

In [70]:
print("Partial: {0}".format(partial))
print("Ratio Type: {0}".format(ratio_type))

Partial: False
Ratio Type: sort


#### Admin Items

#### MMIC Data

In [71]:
# %%time
# item_all_sales_filter = read_r("/data/ALL_MARKETS/MODEL_DATA/item_all_sales_filter.rds")[None]

# item_all_sales_filter = item_all_sales_filter.loc[:, ['COUNTRY_CODE', 'MDS_FAM_ID', 'ITEM_NBR','UPC_NBR','ITEM1_DESC', 'ITEM2_DESC']]

# %%time
# report_all = read_r("/data/ALL_MARKETS/REPORTS/report_all.rds")[None]

# report_all.columns = [col.upper() for col in report_all.columns]

# report_all = report_all.loc[:, ['MDS_FAM_ID', 'COUNTRY_CODE', 'SBU_CLASF_DESC', 'SRC_PRCUR_CLASS_DESC',  'VRTY_PRED_1_DESC',  'PROPOSED_CATG_PRED_1_DESC']]

# %%time
# report_all_item_desc = report_all.merge(item_all_sales_filter, on = ['COUNTRY_CODE', 'MDS_FAM_ID'])

# report_all_item_desc.head()

# with open("/data/sandbox/adhoc-tasks/admin_items_merlin/datasets/report_all_item_desc.pickle", 'wb') as file:
#     pickle.dump(report_all_item_desc,file)

In [72]:
%%time
with open("/data/sandbox/adhoc-tasks/admin_items_merlin/datasets/report_all_item_desc.pickle", 'rb') as file:
    report_all_item_desc = pickle.load(file)

CPU times: user 19 s, sys: 7.07 s, total: 26 s
Wall time: 26 s


In [73]:
report_all_item_desc.shape

(14239432, 10)

In [74]:
report_all_item_desc.head()

,MDS_FAM_ID,COUNTRY_CODE,SBU_CLASF_DESC,SRC_PRCUR_CLASS_DESC,VRTY_PRED_1_DESC,PROPOSED_CATG_PRED_1_DESC,ITEM_NBR,UPC_NBR,ITEM1_DESC,ITEM2_DESC
0,83645009,US,APPAREL,INTIMATES,SLEEP PANTS,SLEEPWEAR,550375790,76705267587,GRAPHIC SLEEP SHIRT,ONLINE ONLY
1,83325257,US,APPAREL,INTIMATES,SLEEP PANTS,SLEEPWEAR,550351020,64600702080,"NB PANT, NZ22106BB",ONLINE ONLY
2,92058933,US,APPAREL,INTIMATES,SLEEP SHORTS,SLEEPWEAR,552607763,76361844922,SLEEPWEAR SETS,ONLINE ONLY
3,92058781,US,APPAREL,INTIMATES,SLEEP SHORTS,SLEEPWEAR,552607634,76361844879,SLEEPWEAR SETS,ONLINE ONLY
4,92059040,US,APPAREL,INTIMATES,SLEEP SHORTS,SLEEPWEAR,552607858,76361844976,SLEEPWEAR SETS,ONLINE ONLY


### FUZZY MATCH WITH STORE ITEM DESCRIPTIONS

In [75]:
with open("/data/sandbox/adhoc-tasks/admin_items_merlin/datasets/All Merlin Items-with SP_CLASS_feat_cleaned.pickle", 'rb') as file:
    admin_data = pickle.load(file)

In [8]:
# with open("/data/sandbox/adhoc-tasks/admin_items_merlin/All Admin Items-with SP_CLASS_feat_cleaned.pickle", 'rb') as file:
#     admin_data = pickle.load(file)

In [76]:
admin_data.head(2)

,id,Global Catalog Id,COUNTRY_CODE,SUPER DEPARTMENT,DEPARTMENT,SBU,SP_CLASS,ITEM NAME,ITEM DESCRIPTION,SBU_SP_CLASS,FEAT,FEAT CLEAN,ITEM NAME CLEAN,ITEM DESCRIPTION CLEAN
0,ADMIN_ec2e55ba-cf09-4dc2-a5b1-449688490525,ec2e55ba-cf09-4dc2-a5b1-449688490525,CA,Home/Seasonal/Toys,Seasonal,ENTERTAINMENT TOYS AND SEASONAL,SEASONAL,Green Dragon Adult Costume,Available in size: S-XL,ENTERTAINMENT TOYS AND SEASONAL_SEASONAL,Green Dragon Adult Costume Available in size: S-XL,GREEN DRAGON ADULT COSTUME AVAILABLE IN SIZE S XL,GREEN DRAGON ADULT COSTUME,AVAILABLE IN SIZE S XL
1,ADMIN_e29f8e09-d1f7-4f25-9c6a-b596e0171edd,e29f8e09-d1f7-4f25-9c6a-b596e0171edd,US,Home/Seasonal/Toys,Seasonal,ENTERTAINMENT TOYS AND SEASONAL,SEASONAL,Easter Grey Bunny Basket,"<p class=""MsoNormal""><span lang=""EN-US"">GREY BUNNY BASKET<o:p></o:p></span></p><p class=""MsoNormal""><ul><li>SIZE : 7.75 DIA X 12.5 H INCH WITH HANDLE&nbsp;</li><li>Care Instructions, Surface wash only<br></li></ul></p>",ENTERTAINMENT TOYS AND SEASONAL_SEASONAL,"Easter Grey Bunny Basket <p class=""MsoNormal""><span lang=""EN-US"">GREY BUNNY BASKET<o:p></o:p></span></p><p class=""MsoNormal""><ul><li>SIZE : 7.75 DIA X 12.5 H INCH WITH HANDLE&nbsp;</li><li>Care Instructions, Surface wash only<br></li></ul></p>",EASTER GREY BUNNY BASKET SIZE 7.75 DIA X 12.5 H INCH WITH HANDLE NBSP CARE INSTRUCTION SURFACE WASH,EASTER GREY BUNNY BASKET,GREY BUNNY BASKET SIZE 7.75 DIA X 12.5 H INCH WITH HANDLE NBSP CARE INSTRUCTION SURFACE WASH


In [77]:
admin_data.shape

(88809, 14)

In [78]:
feat_col = config.feat_col
print(feat_col)

ITEM NAME


In [79]:
sbu_sp = admin_data.loc[:, ['SBU', 'SP_CLASS']].drop_duplicates().reset_index(drop=True)

In [12]:
# import multiprocessing as mp
# pool = mp.Pool(mp.cpu_count())
# print(pool)

In [13]:
%%time
master_df_list = []
failed_list = []
for idx, row in list(sbu_sp.iterrows()):
    
    try:
        sbu, sp_class = row['SBU'], row['SP_CLASS']

        print("====== SBU : {0} || SP_CLASS: {1}======".format(sbu,sp_class))


        admin_data_sub = admin_data.loc[(admin_data['SBU']==sbu) & (admin_data['SP_CLASS']==sp_class), :]
        report_all_sub = report_all_item_desc.loc[(report_all_item_desc['SBU_CLASF_DESC']==sbu) & (report_all_item_desc['SRC_PRCUR_CLASS_DESC']==sp_class), :]

        print("Admin data shape: {0}".format(admin_data_sub.shape))

        list_store = report_all_sub['ITEM1_DESC'].drop_duplicates()

        print("Store data shape: {0}".format(list_store.shape))


        # Run the fuzzy string matching algorithm
        start = time.time()
        df_result = (admin_data_sub.pipe(fuzzy_tf_idf_1, # Function and messy data
                             column = feat_col, # Messy column in data
                             clean = list_store, # Master data (list)
                             mapping_df = admin_data_sub, # Master data
                             col = 'Result', # Can be customized
                             partial = partial, 
                             ratio_type = ratio_type) 
                    )
        end = time.time()
        # Print the computation time
        print('Fuzzy string matching for {0}_{1} in {2} seconds'.format(sbu,sp_class, end - start))
        master_df_list.append(df_result)
        
    except:
        failed_list.append(f"{row['SBU']}_{row['SP_CLASS']}")
        
    

====== SBU : ENTERTAINMENT TOYS AND SEASONAL || SP_CLASS: SEASONAL======
Admin data shape: (37256, 14)
Store data shape: (252287,)
Fuzzy string matching for ENTERTAINMENT TOYS AND SEASONAL_TOYS in 336.3881793022156 seconds
====== SBU : HARDLINES || SP_CLASS: SPORTING GOODS & AUTOMOTIVE======
Admin data shape: (6277, 14)
Store data shape: (518062,)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/IPython/core/magics/execution.py", line 1238, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 37, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/IPython/core/ultratb.p

KeyboardInterrupt: 

In [17]:
mapped_df=pd.concat(master_df_list)

In [17]:
with open(f"/data/sandbox/adhoc-tasks/admin_items_merlin/datasets/fuzzy_matching/item_name_mapped_df_{partial}_{ratio_type}.pickle", 'wb') as file:
    pickle.dump(mapped_df,file)

In [13]:
print(failed_list)

['ENTERTAINMENT TOYS AND SEASONAL_SEASONAL', 'ENTERTAINMENT TOYS AND SEASONAL_TOYS']


In [26]:
# choices = {'a':"Atlanta Falcons", 'b':"New York Jets", 'c':"New York Giants", 'd':"Dallas Cowboys"}
# process.extract("new york jets", choices, limit=3)
# #  [('New York Jets', 100), ('New York Giants', 78)]
# # ("Dallas Cowboys", 90)

In [80]:
with open(f"/data/sandbox/adhoc-tasks/admin_items_merlin/datasets/fuzzy_matching/item_name_mapped_df_{partial}_{ratio_type}.pickle", 'rb') as file:
    mapped_df = pickle.load(file)

In [81]:
mapped_df.shape

(88809, 3)

In [82]:
mapped_df.head()

,ITEM NAME,Result,Ratio
index,,,
0,Green Dragon Adult Costume,GREEN DRAGON,63
1,Easter Grey Bunny Basket,GREY BUNNY BASKET,83
2,Halloween Wand Sai,HALLOWEEN WAND MOON,81
3,George GLITTER SKELETON,GRG GLITTER SKELETON,74
4,George CANDLE STICK,HANGING CANDLE STICK,77


Preserve the index

In [83]:
mapped_df.reset_index(inplace=True)

#### ITEM DECSRIPTIONS WITH ONE-TO-ONE Mapping with CROSS-VARIETY

In [84]:
%%time
report_all_item_desc['CROSS_VARIETY'] = report_all_item_desc['PROPOSED_CATG_PRED_1_DESC'] + "_" +report_all_item_desc['VRTY_PRED_1_DESC']

CPU times: user 2.57 s, sys: 1.55 s, total: 4.12 s
Wall time: 4.08 s


In [85]:
report_all_item_desc.head()

,MDS_FAM_ID,COUNTRY_CODE,SBU_CLASF_DESC,SRC_PRCUR_CLASS_DESC,VRTY_PRED_1_DESC,PROPOSED_CATG_PRED_1_DESC,ITEM_NBR,UPC_NBR,ITEM1_DESC,ITEM2_DESC,CROSS_VARIETY
0,83645009,US,APPAREL,INTIMATES,SLEEP PANTS,SLEEPWEAR,550375790,76705267587,GRAPHIC SLEEP SHIRT,ONLINE ONLY,SLEEPWEAR_SLEEP PANTS
1,83325257,US,APPAREL,INTIMATES,SLEEP PANTS,SLEEPWEAR,550351020,64600702080,"NB PANT, NZ22106BB",ONLINE ONLY,SLEEPWEAR_SLEEP PANTS
2,92058933,US,APPAREL,INTIMATES,SLEEP SHORTS,SLEEPWEAR,552607763,76361844922,SLEEPWEAR SETS,ONLINE ONLY,SLEEPWEAR_SLEEP SHORTS
3,92058781,US,APPAREL,INTIMATES,SLEEP SHORTS,SLEEPWEAR,552607634,76361844879,SLEEPWEAR SETS,ONLINE ONLY,SLEEPWEAR_SLEEP SHORTS
4,92059040,US,APPAREL,INTIMATES,SLEEP SHORTS,SLEEPWEAR,552607858,76361844976,SLEEPWEAR SETS,ONLINE ONLY,SLEEPWEAR_SLEEP SHORTS


In [86]:
%%time
cross_vrty_counts=report_all_item_desc.groupby('ITEM1_DESC').agg({'CROSS_VARIETY' : 'nunique'})

CPU times: user 23.1 s, sys: 1.16 s, total: 24.2 s
Wall time: 24.1 s


In [87]:
cross_vrty_counts.reset_index(inplace=True)

In [88]:
cross_vrty_counts.head()

,ITEM1_DESC,CROSS_VARIETY
0,,4
1,PHTOCKE CH/CH DECO,1
2,PHTOCKE CH/VAN DECO,1
3,ROCKCRAWLEREXTREM,1
4,! G PC INFANT WATER,1


Item descriptions with a single Cross-Variety mapped

In [89]:
item_descs_single_vrty=cross_vrty_counts.loc[cross_vrty_counts['CROSS_VARIETY']==1, 'ITEM1_DESC']

In [90]:
item_desc_cross_vrty_mapping = report_all_item_desc.loc[report_all_item_desc['ITEM1_DESC'].isin(item_descs_single_vrty), ['ITEM1_DESC', 'CROSS_VARIETY']].drop_duplicates()

In [91]:
item_desc_cross_vrty_mapping.head()

,ITEM1_DESC,CROSS_VARIETY
0,GRAPHIC SLEEP SHIRT,SLEEPWEAR_SLEEP PANTS
1,"NB PANT, NZ22106BB",SLEEPWEAR_SLEEP PANTS
41,NB TANK&SHORT,SLEEPWEAR_SLEEP SHORTS
51,"NB CHEMISE,NZ22112DD",LADIES ROBES GOWNS AND CHEMISES_LADIES CHEMISES
55,W SLEEVELESS SLEEP 2,SLEEPWEAR_SLEEP PANTS


In [92]:
admin_data_vrty_item=mapped_df.merge(item_desc_cross_vrty_mapping,left_on = 'Result', right_on = 'ITEM1_DESC', how = "left")

In [93]:
admin_data_vrty_item.head()

,index,ITEM NAME,Result,Ratio,ITEM1_DESC,CROSS_VARIETY
0,0,Green Dragon Adult Costume,GREEN DRAGON,63,NaN,NaN
1,1,Easter Grey Bunny Basket,GREY BUNNY BASKET,83,GREY BUNNY BASKET,EASTER_EASTER BASKETS
2,2,Halloween Wand Sai,HALLOWEEN WAND MOON,81,HALLOWEEN WAND MOON,FASHION STATIONERY_HALLOWEEN STATIONERY
3,3,George GLITTER SKELETON,GRG GLITTER SKELETON,74,GRG GLITTER SKELETON,HALLOWEEN OUTDOOR DECOR_HALLOWEEN SKELETONS
4,4,George CANDLE STICK,HANGING CANDLE STICK,77,HANGING CANDLE STICK,UNASSIGNED_UNASSIGNED


In [94]:
admin_data_vrty_item.set_index('index', inplace=True)

In [95]:
admin_data_vrty_item.head()

,ITEM NAME,Result,Ratio,ITEM1_DESC,CROSS_VARIETY
index,,,,,
0,Green Dragon Adult Costume,GREEN DRAGON,63,NaN,NaN
1,Easter Grey Bunny Basket,GREY BUNNY BASKET,83,GREY BUNNY BASKET,EASTER_EASTER BASKETS
2,Halloween Wand Sai,HALLOWEEN WAND MOON,81,HALLOWEEN WAND MOON,FASHION STATIONERY_HALLOWEEN STATIONERY
3,George GLITTER SKELETON,GRG GLITTER SKELETON,74,GRG GLITTER SKELETON,HALLOWEEN OUTDOOR DECOR_HALLOWEEN SKELETONS
4,George CANDLE STICK,HANGING CANDLE STICK,77,HANGING CANDLE STICK,UNASSIGNED_UNASSIGNED


In [96]:
admin_data = admin_data.join(admin_data_vrty_item.drop('ITEM NAME', axis = "columns"))

In [97]:
with open(f"/data/sandbox/adhoc-tasks/admin_items_merlin/datasets/fuzzy_matching/all_merlin_items_fuzzy_matching_{partial}_{ratio_type}.pickle", 'wb') as file:
    pickle.dump(admin_data, file)

In [98]:
# with open(f"/data/sandbox/adhoc-tasks/admin_items_merlin/datasets/fuzzy_matching/admin_items_fuzzy_matching_{partial}_{ratio_type}.pickle", 'wb') as file:
#     pickle.dump(admin_data, file)

In [99]:
admin_data.head()

,id,Global Catalog Id,COUNTRY_CODE,SUPER DEPARTMENT,DEPARTMENT,SBU,SP_CLASS,ITEM NAME,ITEM DESCRIPTION,SBU_SP_CLASS,FEAT,FEAT CLEAN,ITEM NAME CLEAN,ITEM DESCRIPTION CLEAN,Result,Ratio,ITEM1_DESC,CROSS_VARIETY
0,ADMIN_ec2e55ba-cf09-4dc2-a5b1-449688490525,ec2e55ba-cf09-4dc2-a5b1-449688490525,CA,Home/Seasonal/Toys,Seasonal,ENTERTAINMENT TOYS AND SEASONAL,SEASONAL,Green Dragon Adult Costume,Available in size: S-XL,ENTERTAINMENT TOYS AND SEASONAL_SEASONAL,Green Dragon Adult Costume Available in size: S-XL,GREEN DRAGON ADULT COSTUME AVAILABLE IN SIZE S XL,GREEN DRAGON ADULT COSTUME,AVAILABLE IN SIZE S XL,GREEN DRAGON,63,NaN,NaN
1,ADMIN_e29f8e09-d1f7-4f25-9c6a-b596e0171edd,e29f8e09-d1f7-4f25-9c6a-b596e0171edd,US,Home/Seasonal/Toys,Seasonal,ENTERTAINMENT TOYS AND SEASONAL,SEASONAL,Easter Grey Bunny Basket,"<p class=""MsoNormal""><span lang=""EN-US"">GREY BUNNY BASKET<o:p></o:p></span></p><p class=""MsoNormal""><ul><li>SIZE : 7.75 DIA X 12.5 H INCH WITH HANDLE&nbsp;</li><li>Care Instructions, Surface wash only<br></li></ul></p>",ENTERTAINMENT TOYS AND SEASONAL_SEASONAL,"Easter Grey Bunny Basket <p class=""MsoNormal""><span lang=""EN-US"">GREY BUNNY BASKET<o:p></o:p></span></p><p class=""MsoNormal""><ul><li>SIZE : 7.75 DIA X 12.5 H INCH WITH HANDLE&nbsp;</li><li>Care Instructions, Surface wash only<br></li></ul></p>",EASTER GREY BUNNY BASKET SIZE 7.75 DIA X 12.5 H INCH WITH HANDLE NBSP CARE INSTRUCTION SURFACE WASH,EASTER GREY BUNNY BASKET,GREY BUNNY BASKET SIZE 7.75 DIA X 12.5 H INCH WITH HANDLE NBSP CARE INSTRUCTION SURFACE WASH,GREY BUNNY BASKET,83,GREY BUNNY BASKET,EASTER_EASTER BASKETS
2,ADMIN_fea2b9da-172b-4641-bfdb-90cd7e011df2,fea2b9da-172b-4641-bfdb-90cd7e011df2,US,Home/Seasonal/Toys,Seasonal,ENTERTAINMENT TOYS AND SEASONAL,SEASONAL,Halloween Wand Sai,Halloween Wand Sai,ENTERTAINMENT TOYS AND SEASONAL_SEASONAL,Halloween Wand Sai Halloween Wand Sai,HALLOWEEN WAND SAI,HALLOWEEN WAND SAI,HALLOWEEN WAND SAI,HALLOWEEN WAND MOON,81,HALLOWEEN WAND MOON,FASHION STATIONERY_HALLOWEEN STATIONERY
3,ADMIN_137c230f-3599-4d47-b819-2d4c17f88da8,137c230f-3599-4d47-b819-2d4c17f88da8,GB,Home/Seasonal/Toys,Seasonal,ENTERTAINMENT TOYS AND SEASONAL,SEASONAL,George GLITTER SKELETON,This is an IMPORT Item,ENTERTAINMENT TOYS AND SEASONAL_SEASONAL,George GLITTER SKELETON This is an IMPORT Item,GEORGE GLITTER SKELETON THIS IS AN IMPORT,GEORGE GLITTER SKELETON,THIS IS AN IMPORT,GRG GLITTER SKELETON,74,GRG GLITTER SKELETON,HALLOWEEN OUTDOOR DECOR_HALLOWEEN SKELETONS
4,ADMIN_ffb10a7f-685e-4dfa-bda4-07f8a28fb93b,ffb10a7f-685e-4dfa-bda4-07f8a28fb93b,GB,Home/Seasonal/Toys,Seasonal,ENTERTAINMENT TOYS AND SEASONAL,SEASONAL,George CANDLE STICK,This is an IMPORT Item,ENTERTAINMENT TOYS AND SEASONAL_SEASONAL,George CANDLE STICK This is an IMPORT Item,GEORGE CANDLE STICK THIS IS AN IMPORT,GEORGE CANDLE STICK,THIS IS AN IMPORT,HANGING CANDLE STICK,77,HANGING CANDLE STICK,UNASSIGNED_UNASSIGNED
